In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import precision_score, f1_score, confusion_matrix

In [ ]:
transform = transforms.Compose({
    transforms.Resize((224, 224)),
    transforms.ToTensor()
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Test2',transform= transform)

dataset.classes

['Fake', 'Real']

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
from torchvision import models

vgg16 = models.vgg16(pretrained=True)
vgg16

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 226MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch.nn as nn
import torch.optim as optim

vgg16.classifier[6] = nn.Linear(4096, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


vgg16.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        outputs = vgg16(images)
        _, predicted = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(predicted.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"Confusion Matris: \n{conf_matrix}")

Epoch 1, Loss: 0.9532813213088296
Epoch 2, Loss: 0.6988236470655962
Epoch 3, Loss: 0.7152152467857708
Precision: 0.0
F1 Score: 0.0
Confusion Matris: 
[[82  0]
 [92  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
for param in vgg16.features[-4:].parameters():
    param.requires_grad = True

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


vgg16.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        outputs = vgg16(images)
        _, predicted = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(predicted.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"Confusion Matris: \n{conf_matrix}")

Epoch 1, Loss: 0.8549018987902889
Epoch 2, Loss: 0.7031187922866257
Epoch 3, Loss: 0.7238096683113663
Precision: 0.5380952380952381
F1 Score: 0.6996904024767802
Confusion Matris: 
[[  0  97]
 [  0 113]]


In [ ]:
vgg16.features = nn.Sequential(
    vgg16.features,
    nn.Conv2d(512, 512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
)

vgg16.classifier[6] = nn.Linear(512 * 7 * 7, 2)


In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


vgg16.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        outputs = vgg16(images)
        _, predicted = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(predicted.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"Confusion Matris: \n{conf_matrix}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x4096 and 25088x2)

In [ ]:
from transformers import ViTForImageClassification, ViTFeatureExtractor

# Load Vision Transformer model
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
vit_model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(dataset.classes)
)


# Preprocess dataset
def preprocess(images):
    return feature_extractor(images=images, return_tensors="pt")['pixel_values']

# Adjust training loop for ViT
vit_optimizer = optim.Adam(vit_model.parameters(), lr=0.001)
for epoch in range(num_epochs):
    vit_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        pixel_values = preprocess(images)
        vit_optimizer.zero_grad()
        outputs = vit_model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()
        vit_optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
